In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [64]:

df=pd.read_csv(r'C:\Users\DO TUAN MINH\Desktop\ben\Learn\Project\demo_data\final.csv')

In [65]:
df=df.drop(columns=['D THU'])

In [66]:
df = df.dropna().reset_index(drop=True)


In [67]:
x=df.drop(columns=['revenue','date'])
y=df['revenue']


In [68]:
weekday_columns = ['weekday_Monday', 'weekday_Tuesday', 'weekday_Wednesday', 'weekday_Thursday', 'weekday_Friday', 'weekday_Saturday', 'weekday_Sunday']
df[weekday_columns] = df[weekday_columns].astype(int)


In [81]:
df.count

<bound method DataFrame.count of            date  weekday_Friday  weekday_Monday  weekday_Saturday  \
0    2023-01-08               0               0                 0   
1    2023-01-09               0               1                 0   
2    2023-01-10               0               0                 0   
3    2023-01-11               0               0                 0   
4    2023-01-12               0               0                 0   
..          ...             ...             ...               ...   
308  2023-12-26               0               0                 0   
309  2023-12-28               0               0                 0   
310  2023-12-29               1               0                 0   
311  2023-12-30               0               0                 1   
312  2023-12-31               0               0                 0   

     weekday_Sunday  weekday_Thursday  weekday_Tuesday  weekday_Wednesday  \
0                 1                 0                0       

In [54]:
df.dtypes

date                       object
weekday_Friday              int32
weekday_Monday              int32
weekday_Saturday            int32
weekday_Sunday              int32
weekday_Thursday            int32
weekday_Tuesday             int32
weekday_Wednesday           int32
totalprecip_mm            float64
avgvis_km                 float64
temp_c                    float64
wind_kph                  float64
pressure_in               float64
humidity                    int64
revenue                   float64
revenue_prev1             float64
revenue_prev2             float64
revenue_mean_3            float64
revenue_mean_7            float64
revenue_diff              float64
revenue_pct_change        float64
revenue_dev_from_mean7    float64
dtype: object

In [71]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test =train_test_split(x,y, test_size=0.2, shuffle=False, random_state=42)

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Lọc các cột số, trừ cột target 'revenue' và các cột one-hot weekday
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
exclude_columns = ['revenue'] + ['weekday_Monday', 'weekday_Tuesday', 'weekday_Wednesday',
                                 'weekday_Thursday', 'weekday_Friday', 'weekday_Saturday', 'weekday_Sunday']
numeric_columns = [col for col in numeric_columns if col not in exclude_columns]

# Chia dữ liệu thành train/test
X = df.drop(columns=['revenue', 'date'])  # X là các đặc trưng (features)
y = df['revenue']  # y là target (doanh thu)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tạo đối tượng StandardScaler và chỉ fit trên dữ liệu huấn luyện
scaler = StandardScaler()

# Fit scaler trên tập huấn luyện và transform trên cả tập huấn luyện và kiểm tra


X_train_scaled = scaler.fit_transform(X_train[numeric_columns])
X_test_scaled = scaler.transform(X_test[numeric_columns])

# Chuyển lại thành DataFrame với cột giống như ban đầu
X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=numeric_columns, index=X_test.index)


# Gộp lại với các cột không phải số
X_train_final = X_train.drop(columns=numeric_columns).join(X_train_scaled)
X_test_final = X_test.drop(columns=numeric_columns).join(X_test_scaled)

# Kiểm tra dữ liệu sau khi scale
print(X_train_final.head())
print(X_test_final.head())


     weekday_Friday  weekday_Monday  weekday_Saturday  weekday_Sunday  \
90                0               0                 0               0   
218               0               0                 1               0   
256               0               0                 0               0   
137               0               0                 0               0   
204               0               0                 0               0   

     weekday_Thursday  weekday_Tuesday  weekday_Wednesday  totalprecip_mm  \
90                  0                0                  1       -0.410929   
218                 0                0                  0       -0.140456   
256                 1                0                  0       -0.401708   
137                 1                0                  0       -0.410929   
204                 1                0                  0       -0.189633   

     avgvis_km    temp_c  wind_kph  pressure_in  humidity  revenue_prev1  \
90    0.595958  1.3458

In [80]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor

import pandas as pd
import numpy as np

# Danh sách mô hình và tên
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='linear')
}

# Data đầu vào: X_train_final, y_train, X_test_final, y_test

results = []

for name, model in models.items():
    model.fit(X_train_final, y_train)
    y_pred = model.predict(X_test_final)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({
        'Model': name,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    })

# In kết quả
results_df = pd.DataFrame(results).sort_values(by='R2', ascending=False)
print(results_df)


              Model         MAE        RMSE        R2
0  LinearRegression  436.162159  546.364395  0.652146
1             Ridge  438.775808  547.654524  0.650502
2             Lasso  437.418333  548.707244  0.649157
3      RandomForest  466.875238  610.431973  0.565784
4           XGBoost  508.054490  671.398645  0.474718
5               SVR  717.504050  877.028751  0.103688
